*Применение рекуррентных нейронных сетей (recurrent neural networks) в задачах распознавания естественного языка. Детали архитектуры LSTM.*

---

# В предыдущей серии...

![text](attach1.png) ![text](attach2.png) ![text](attach3.png)

*(если не понятно, что здесь происходит, стоит вернуться к прошлой лекции!)*

Универсальные и гибкие, часто используются в NLP, тренируемые... конечно же, это

# Рекуррентные нейронные сети

*По мотивам почта The Unreasonable Effectiveness of Recurrent Neural Networks*

Как говорилось в прошлой лекции, в самом NLP очень много разнообразных юзкейсов — они описаны на картинке ниже (красный — входные слова, синий — выходные слова):

![alt text](attach4.png)

- One to One (ближе к классической нейронной сети — вход и выход фиксированного размера)
- One to Many (с этим и другими видами ниже мы и познакомимся)
- Many to One (например, анализ характера предложения)
- Many to Many (например, перевод текстов)

Но как должна выглядеть сеть, которая может работать с переменным количеством данных на вход и выход?

# Архитектура

Возьмём какой-то один линейный слой:

![alt text](attach5.png)

И начнём **выход** этого слоя давать на **вход**, помимо входных данных, следующему слою:

![alt text](attach6.png)

То есть у такой сети есть возможность передать себе в "будущее" некоторые состояние — и мы тренируем такую систему, где сеть выучивается, как некоторым образом закодировать и передать на следующем шаге, чтобы "знать" результаты прошлых шагов — и иметь доступ ко всей цепочке последовательности.

Получается такая своего рода рекурсия — поэтому сеть и называется **Recurrent Neural Network (рекурсивная нейроннасть сеть)**. Реализация в коде выглядит так:

![alt text](attach7.png)

Другое представление этого же принципа работы, из блогпоста А. Карпатого:

![alt text](attach8.png)

![alt text](attach9.png)

# Пример: генерация текста

### Training

Разобьём весь наш текст на последовательность символов. Также добавим специальные символы, означаютщие, например, о начале и конце строки:

```
Hello
--->
'[НАЧАЛО_СТРОКИ]' 'H' 'e' 'l' 'l' 'o'
--->
'H' 'e' 'l' 'l' 'o' '[КОНЕЦ_СТРОКИ]'
```

Как выглядит наш словарь для примера выше:

![alt text](attach12.png)

То есть сеть тренируется по прошлым символам (являются контекстом) предсказывать следующий символ. Ей нужно обучиться, какие символы чаще всего следуют за другими.

### Inference

Например, подадим на вход спецсимвол начала сткрои и посмотрим, что будет дальше:

```
'[НАЧАЛО_СТРОКИ]' -> 'H'
--->
'H' -> 'e'
--->
'e' -> ...
--->
... -> '[КОНЕЦ_СТРОКИ]'
```

... и, если делать train достаточно долго, даже такая несложная, генерирующая посимвольно, простая модель даёт относительно связный текст! Из минусов — в отличии от word2vec, с такой моделью Transfer Learning работает крайне плохо.

![alt text](attach11.png)

# Long Short-Term Memory (LSTM), 1997

Но есть небольшой секрет в этом прекрасном мире... Если написать код, как в примере выше, сеть работать НЕ БУДЕТ:

![alt text](attach7.png)

### Но почему?

![alt text](attach13.png)

![alt text](attach14.png)

### И что же делать?

Делать шаг более по-умному, нежели просто делать умножение на матрицу. Наиболее популярный вариант такой реализации называется **Long Short-Term Memory**.

Архитектура, которую мы разбирали до этого, называется **Vanilla RNN**:

![alt text](attach8.png)

А вот так страшненько выглядит наш спаситель:

![alt text](attach16.png)


### Как ОНО работает?

Самое важное изменение — мы передаём не один вектор, а два — помимо hidden layer ($h$), мы будем передавать self-state ($C$). 

И если в вычислении первого ничего принципиально не меняется (т.е. он всё так же будет подвержен тем же проблемам), то в последнем мы будем делать изменения очень осторожно, чтобы градиенты по нему всегда протекали хорошо из прошлого шага в следующий.

![alt text](attach15.png)

Помимо этого, мы введём gate-ы ($\sigma = \frac{1}{1+e^{-x}}$) — различные вектора коэффициентов, на которые мы будем перемножать некоторые входные значения:

- Первым таким gate-ом будет **Forget gate**. Он выдаёт вектор от 0 до 1 для каждой из компонент, который говорит для каждой из них, сколько из неё стоит забыть;
- Вторым таким gate-ом будем **Input gate**. Он выдаёт вектор от 0 до 1 для каждой из компонент, на сколько стоит домножить добавку этого шага к $c$;
- Затем происходит **Cell Update**. На выход self-state шага пойдёт $C_t$ (self-state после всех манипуляциями).
- Наконец, идёт **Output Gate**. Он говорит, каким образом мы получаем $h_t$ (hidden layer после всех манипуляций, он пойдёт на выход).

### Почему ОНО работает?

![alt text](attach17.png)

Для $C$ — взяли с прошлого, умножили на forget gate, добаивли добавку и пустили дальше. Получился такой типа хайвэй, на котором градиент (справа сверху обозначен стрелкою) протекает намного лучше и меньше затухает.

# Варианты LSTM

На текущий\* момент, существует много разных модификаций LSTM, например с Peephole Connections или No Input Gate.

Но как правило, используют либо обычный LSTM, либо GRU (версия LSTM от Google с упрощёнными вычислениями):

![alt text](attach18.png)

Также, до изобретения LSTM, люди часто использовали Bidirectional RNN — рекурсивная нейросеть и отдельными словами для forward и backward проходом. Но, для задач типа генерации текста она не подходит. В современном мире такой подод исопльзуется в модификации Bidirectional LSTM для случаев, когда нужен очень очень хорошо протексающий по огромному числу слоёв и не затухающий в процессе градиент.

# LSTM (иногда + word2vec) — очень эффективная комбинация для решения задач NLP!

![alt text](attach19.png)

### Более "низкоуровненый" разбор работы на примере Part of Speech Tagging:

![alt text](attach20.png)

*В следующий раз будет про ЕЩЁ более крутые инструменты для решения проблем, близкие к SoTA на момент 2019.*

***А пока — пока!***